Download dataset from https://www.kaggle.com/sobhanmoosavi/us-weather-events
extract the csv file and rename it to weather.csv
run following code to explore the dataset

In [130]:
import pandas as pd
import numpy as np
import string
import random
import datetime

In [2]:
weather = pd.read_csv("./weather.csv")

In [3]:
weather.head(5)

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


In [4]:
len(weather)

6274206

In [5]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6274206 entries, 0 to 6274205
Data columns (total 13 columns):
 #   Column          Dtype  
---  ------          -----  
 0   EventId         object 
 1   Type            object 
 2   Severity        object 
 3   StartTime(UTC)  object 
 4   EndTime(UTC)    object 
 5   TimeZone        object 
 6   AirportCode     object 
 7   LocationLat     float64
 8   LocationLng     float64
 9   City            object 
 10  County          object 
 11  State           object 
 12  ZipCode         float64
dtypes: float64(3), object(10)
memory usage: 622.3+ MB


In [6]:
weather['StartTime(UTC)'] = pd.to_datetime(weather['StartTime(UTC)'])
weather['EndTime(UTC)'] = pd.to_datetime(weather['EndTime(UTC)'])
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6274206 entries, 0 to 6274205
Data columns (total 13 columns):
 #   Column          Dtype         
---  ------          -----         
 0   EventId         object        
 1   Type            object        
 2   Severity        object        
 3   StartTime(UTC)  datetime64[ns]
 4   EndTime(UTC)    datetime64[ns]
 5   TimeZone        object        
 6   AirportCode     object        
 7   LocationLat     float64       
 8   LocationLng     float64       
 9   City            object        
 10  County          object        
 11  State           object        
 12  ZipCode         float64       
dtypes: datetime64[ns](2), float64(3), object(8)
memory usage: 622.3+ MB


In [7]:
start2020 = pd.to_datetime("2020-01-01")
weather = weather[weather['StartTime(UTC)'] >= start2020]

In [8]:
weather.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
1721,W-1722,Snow,Light,2020-01-01 19:30:00,2020-01-01 22:12:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1722,W-1723,Snow,Light,2020-01-02 00:32:00,2020-01-02 00:50:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1723,W-1724,Snow,Light,2020-01-02 01:12:00,2020-01-02 01:30:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1724,W-1725,Snow,Light,2020-01-02 04:12:00,2020-01-02 04:30:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1725,W-1726,Snow,Light,2020-01-02 05:30:00,2020-01-02 06:12:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


In [9]:
weather.reset_index(drop=True, inplace=True)
weather.drop(columns=['EventId', 'EndTime(UTC)', 'AirportCode', 'LocationLat', 'LocationLng', 'County', 'ZipCode'], inplace=True)
weather.rename(columns={"StartTime(UTC)":"Date"}, inplace=True)
# remove the hour:minute:second from Date column
weather['Date'] = weather['Date'].map(lambda r: r.date())

In [19]:
print(weather.head())
print(len(weather))

   Type Severity        Date     TimeZone      City State
0  Snow    Light  2020-01-01  US/Mountain  Saguache    CO
1  Snow    Light  2020-01-02  US/Mountain  Saguache    CO
2  Snow    Light  2020-01-02  US/Mountain  Saguache    CO
3  Snow    Light  2020-01-02  US/Mountain  Saguache    CO
4  Snow    Light  2020-01-02  US/Mountain  Saguache    CO
1214221


In [26]:
# randomly select some record, we only need approximately 5k record for this project
simplerW = weather.sample(n=10000)
a = simplerW['Type'].unique()
print(len(a))
b = simplerW[['City', 'State']].drop_duplicates()
print(len(b))
c = simplerW[['Date', 'Severity', 'Type', 'City', 'State']].drop_duplicates()
print(len(c))

7
1792
9876


In [98]:
# WeatherType table
d1 = simplerW['Type'].unique()
p1 = pd.DataFrame(d1, columns=['TypeName'])
p1.insert(0, 'TypeId', range(1, len(d1) + 1))
p1.reset_index(drop=True, inplace=True)

print(p1)
p1.to_csv('WeatherType.csv', index=False, header=False)

   TypeId       TypeName
0       1           Rain
1       2            Fog
2       3           Cold
3       4           Snow
4       5          Storm
5       6  Precipitation
6       7           Hail


In [99]:
# Location table
d2 = simplerW[['City', 'State']].drop_duplicates()

## remove invalide rows. Any cell having empty string is consider invalid
d2['City'].replace('', np.nan, inplace=True)
d2['State'].replace('', np.nan, inplace=True)
d2.dropna(subset=['City', 'State'], inplace=True)

d2.insert(0, 'LocationId', range(1, len(d2) + 1))
d2.reset_index(drop=True, inplace=True)

print(d2.head())
d2.to_csv('Location.csv', index=False, header=False)

   LocationId           City State
0           1     Statesboro    GA
1           2           Alva    OK
2           3        Brumley    MO
3           4  New Braunfels    TX
4           5      Blue Bell    PA


In [125]:
d3 = simplerW[['Date', 'Severity', 'Type', 'City', 'State']].drop_duplicates()
d3.insert(0, 'EventId', range(1, len(d3) + 1))
d3.reset_index(drop=True, inplace=True)
d3.insert(1, 'LocationId', np.nan)
d3.insert(1, 'WeatherTypeId', np.nan)

## replace Type, City, State with WeatherTypeId and LocationId
for index, row in d3.iterrows():
    try:
        d3.at[index, 'WeatherTypeId'] = p1[p1['TypeName'] == row['Type']].iloc[0]['TypeId']
        d3.at[index, 'LocationId'] = d2[d2['City'] == row['City']].iloc[0]['LocationId']
    except IndexError:
        pass

## remove unnecessary column
d3.drop(['Type', 'City', 'State'], axis=1, inplace=True)

## remove invalid rows
d3.dropna(subset=['WeatherTypeId', 'LocationId'], inplace=True)

## convert column to correct type
d3['WeatherTypeId'] = pd.to_numeric(d3['WeatherTypeId'], downcast='unsigned')
d3['LocationId'] = pd.to_numeric(d3['LocationId'], downcast='unsigned')

print(d3.head())
d3.to_csv('Event.csv', index=False, header=False)

   EventId  WeatherTypeId  LocationId        Date Severity
0        1              1           1  2020-06-15    Light
1        2              2           2  2020-10-12   Severe
2        3              1           3  2020-06-27    Light
3        4              2           4  2020-03-08   Severe
4        5              1           5  2020-04-15    Light


In [127]:
userNumber = 1001

In [128]:
# UserProfile 
x = [['Team 24', -5, 0]] # Name, Timezone, LocationId
letters = string.ascii_letters

## randomly generate data
for i in range(1, userNumber):
    name = ''.join(random.choice(letters) for i in range(6))
    timezone = random.randint(-11, 12) # based on UTC
    locationId = random.randrange(len(d2))
    x.append([name, timezone, locationId])

d4 = pd.DataFrame(x, columns=['Name', 'Timezone', 'LocationId'])
d4.insert(0, 'UserId', range(1, userNumber + 1))
print(d4.head())
d4.to_csv('UserProfile.csv', index=False, header=False)

   UserId     Name  Timezone  LocationId
0       1  Team 24        -5           0
1       2   jmsmOo         4         238
2       3   LYdnIQ         0         316
3       4   rWcasP        11         578
4       5   SBSOaR         2        1107


In [129]:
# UserCredential table
y = [['root','root']] # username, password
letters = string.ascii_lowercase

## randomly generate data
for i in range(1, userNumber):
    username = ''.join(random.choice(letters) for i in range(6))
    password = ''.join(random.choice(letters) for i in range(8))
    y.append([username, password])

d5 = pd.DataFrame(y, columns=['UserName', 'Password'])
d5.insert(0, 'UserId', range(1, userNumber + 1))

print(d5.head())
d5.to_csv('UserCredential.csv', index=False, header=False)

   UserId UserName  Password
0       1     root      root
1       2   lncvof  qqrernof
2       3   uzooru  gnhxsmph
3       4   iwxslr  pxhvmixt
4       5   dtftgf  rewtzkdr


In [131]:
# UserSearchHistory
z = []

## randomly generate date
startDate = datetime.date(2020, 1, 1)
endDate = datetime.date(2021, 10, 14)

days_between_dates = (endDate - startDate).days

for i in range(1,2000):
    randomUser = random.randrange(1, userNumber + 1)
    randomEvent = random.randrange(1, len(d3) + 1)
    randomDate = startDate + datetime.timedelta(days=random.randrange(days_between_dates))
    z.append([randomUser, randomEvent, randomDate])

d6 = pd.DataFrame(z, columns=['UserId', 'EventId', 'SearchedDate'])

print(d6.head())
d6.to_csv('UserSearchHistory.csv', index=False, header=False)

   UserId  EventId SearchedDate
0     785     2953   2020-04-08
1     830     5386   2020-09-10
2     862     9226   2020-05-02
3     661     9724   2021-08-12
4     887     2261   2021-02-14
